<a href="https://colab.research.google.com/github/schumbar/CMPE297/blob/main/assignment_06/ShawnChumbar_Assignment06_PartA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 06 Part A: Implement NanoGPT

## Assignment Description

This portion of the assignment involves implementing NanoGPT from scratch using PyTorch. After building the implementation, you will train the model using a book of your choice, ensuring it differs from Shakespeare's works, which are used in the original NanoGPT setup.

The final submission should include several deliverables: a Colab notebook demonstrating the code, a Medium article that explains each section of the code in depth, the input and output files used in training, and a short, 10-minute presentation that walks through the implementation step by step. Utilizing the GPT-4 Code Interpreter plugin for assistance is encouraged to streamline the coding process and debugging.

### List of Deliverables:
1. Colab Notebook Demonstrating the code
2. Medium article that explains each section of the code in depth
3. Input files
4. Output files
5. 10-minute video presentation that walks through implementation step by step.

### References:
1. [NanoGPT Colab](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing)
2. [YouTube Tutorial](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=18s)
3. [Class Presentation](https://docs.google.com/presentation/d/1fk8QlODYkBTTH4ftw8M7Sw_tmhJa8KB97s7dYP6s4mI/edit#slide=id.g24535d0c6d4_0_178)
4. ChatGPT Code Interpretter plugin


## Install Packages

In [ ]:
# Clone the nanoGPT repository
!git clone https://github.com/karpathy/nanoGPT.git
%cd nanoGPT

# Install required packages
!pip install numpy transformers datasets tiktoken wandb tqdm

# Upgrade PyTorch to version 2.0.1 with CUDA 11.8 support
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118



Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (682/682), 952.47 KiB | 32.84 MiB/s, done.
Resolving deltas: 100% (385/385), done.
/content/nanoGPT
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages tha

## Check Torch Version

In [ ]:
import torch
print(torch.__version__)

2.0.1+cu118


## Download Shakespeare's Works and Create Data Directory

In [ ]:
# Download Shakespeare's works
!wget https://www.gutenberg.org/files/100/100-0.txt -O shakespeare.txt

# Create a directory for the data
!mkdir data/shakespeare

# Move the text file to the data directory
!mv shakespeare.txt data/shakespeare/input.txt


--2024-11-18 22:29:16--  https://www.gutenberg.org/files/100/100-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5618733 (5.4M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   5.36M  2.66MB/s    in 2.0s    

2024-11-18 22:29:19 (2.66 MB/s) - ‘shakespeare.txt’ saved [5618733/5618733]

mkdir: cannot create directory ‘data/shakespeare’: File exists


## Prepare Data

In [ ]:
# Create the prepare.py script
%%writefile data/shakespeare/prepare.py

import os
import pickle
import numpy as np

# Define input and output paths
input_file_path = 'data/shakespeare/input.txt'
train_output_file_path = 'data/shakespeare/train.bin'
val_output_file_path = 'data/shakespeare/val.bin'

# Read the input text
with open(input_file_path, 'r', encoding='utf-8') as f:
    data = f.read()

# Get all unique characters
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(f"Unique characters: {vocab_size}")

# Create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

# Encode the entire text data
data_size = len(data)
print(f"Data has {data_size} characters.")

# Convert data to integers
encoded_data = np.array([stoi[c] for c in data], dtype=np.uint16)

# Split data into training and validation sets
n = int(0.9 * len(encoded_data))
train_data = encoded_data[:n]
val_data = encoded_data[n:]

# Save the data to .bin files
train_data.tofile(train_output_file_path)
val_data.tofile(val_output_file_path)

# Save the mapping for decoding
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open('data/shakespeare/meta.pkl', 'wb') as f:
    pickle.dump(meta, f)


Overwriting data/shakespeare/prepare.py


In [ ]:
# Run the data preparation script
!python data/shakespeare/prepare.py

Unique characters: 100
Data has 5359439 characters.


In [ ]:
# Create the shakespeare.py config file
%%writefile config/shakespeare.py

# Configuration for training nanoGPT on Shakespeare's works

out_dir = 'out-shakespeare'
eval_interval = 500
eval_iters = 200
log_interval = 100

always_save_checkpoint = False

wandb_log = False  # Set to True if using Weights & Biases
wandb_project = 'shakespeare'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare'
batch_size = 64
block_size = 256  # Context length

# Model configuration
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3
max_iters = 5000
lr_decay_iters = 5000  # Make equal to max_iters
min_lr = 1e-4
beta2 = 0.99

warmup_iters = 100

Writing config/shakespeare.py


## Check if CUDA is Available

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"CUDA is available. GPU device name: {gpu_name}")
else:
    print("CUDA is not available. No GPU detected.")

CUDA is available. GPU device name: NVIDIA A100-SXM4-40GB


## Start Training

In [ ]:
# Start training
!python train.py config/shakespeare.py

Overriding config with config/shakespeare.py:

# Configuration for training nanoGPT on Shakespeare's works

out_dir = 'out-shakespeare'
eval_interval = 500
eval_iters = 200
log_interval = 100

always_save_checkpoint = False

wandb_log = False  # Set to True if using Weights & Biases
wandb_project = 'shakespeare'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare'
batch_size = 64
block_size = 256  # Context length

# Model configuration
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3
max_iters = 5000
lr_decay_iters = 5000  # Make equal to max_iters
min_lr = 1e-4
beta2 = 0.99

warmup_iters = 100

tokens per iteration will be: 655,360
found vocab_size = 100 (inside data/shakespeare/meta.pkl)
Initializing a new model from scratch
number of parameters: 10.66M
num decayed parameter tensors: 26, with 10,753,536 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: True
compiling the model... (takes a ~minute)
step 0: train loss 4.6